In [1]:
import re, csv, datetime, time, lxml, requests, os
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd 
from pandas import DataFrame, Series
import json

#칼럼
columns_news = ['검색어', '게재일', '게재시간', '매체명', '헤드라인', '본문', 'URL', '댓글 수', '좋아요 수', '슬퍼요 수', 
                '화나요 수', '훈훈해요 수', '후속기사 희망 수']
columns_comments = ['URL', '댓글작성자', '댓글', '공감 수', '비공감 수', '댓글작성시간']

#데이터프레임 (기사용, 댓글용)
df_news = pd.DataFrame(columns=columns_news)
df_comments = pd.DataFrame(columns=columns_comments)

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }

# 수집날짜 
s_date = "2020.01.01"
e_date = "2020.01.31"
s_from = s_date.replace(".", "")
e_to = e_date.replace(".", "")
page = 1

#수집 키워드
query = '자가격리'

#기사 수집
def get_news(n_url):    
    news_detail = []
    res = requests.get(n_url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    #게재일 및 게재시간 
    date_day = soup.select('.t11')[0].get_text()[:10]
    news_detail.append(date_day)
    date_time = soup.select('.t11')[0].get_text()[11:]
    news_detail.append(date_time)
    
    #매체명 
    media_outlet = soup.select('div.article_header > div.press_logo')[0].a.img.get('title')
    news_detail.append(media_outlet)

    #헤드라인 
    title = soup.select('h3#articleTitle')[0].text
    news_detail.append(title)

    #본문
    _text = soup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    body_text = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(body_text.strip())
    
    return news_detail


#기사별 공감 지표 수집 - split 활용
def get_like_statics(n_url):
    oid = n_url.split("oid=")[1].split("&")[0]
    aid = n_url.split("aid=")[1]
    header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": n_url
    }

    #Jquery URL 
    c_url = "https://news.like.naver.com/v1/search/contents?suppress_response_codes=true&callback=jQuery112405673248941186477_1550071279893&q=NEWS%5Bne_" + oid + "_" + aid + "%5D%7CNEWS_MAIN%5Bne_" + oid + "_" + aid + "%5D&isDuplication=false&_=1550071279894"
    res = requests.get(c_url, headers=header)
    cont = BeautifulSoup(res.content, "html.parser")

    like_statics = []    

    #좋아요 수치 파싱
    if not 'like","count":' in str(cont):
        like_news = 0
    else:
        like_news = str(cont).split('like","count":')[1].split(",")[0]
    like_statics.append(int(like_news))

    #슬퍼요 수치 파싱
    if not 'sad","count":' in str(cont):
        sad_news = 0
    else:
        sad_news = str(cont).split('sad","count":')[1].split(",")[0]
    like_statics.append(int(sad_news))

    #화나요 수치 파싱
    if not 'angry","count":' in str(cont):
        angry_news = 0
    else:
        angry_news = str(cont).split('angry","count":')[1].split(",")[0]
    like_statics.append(int(angry_news))

    #후속기사 원해요 수치 파싱
    if not 'want","count":' in str(cont):
        want_news = 0
    else:
        want_news = str(cont).split('want","count":')[1].split(",")[0]
    like_statics.append(int(want_news))

    #훈훈해요 수치 파싱
    if not 'warm","count":' in str(cont):
        warm_news = 0
    else:
        warm_news = str(cont).split('warm","count":')[1].split(",")[0]
    like_statics.append(int(warm_news))

    return like_statics


#댓글 수량 수집 - json 파싱 
def get_comment_num(n_url):
    oid = n_url.split("oid=")[1].split("&")[0]
    aid = n_url.split("aid=")[1]
    header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": n_url,
    }

    #댓글 jqeury URL
    c_url ='https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery112409540374534082234_1571293027001&lang=ko&country=KR&objectId=news{}%2C{}&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=20&moveTo=&sort=favorite&includeAllStatus=true&_=1571293027002'.format(oid, aid)
    res = requests.get(c_url, headers=header)
    contents = BeautifulSoup(res.content, "html.parser")
    comments = contents.text.replace('jQuery112409540374534082234_1571293027001(','').replace(');','')
    
    comment_no = []
    
    #json 파일 변환 
    json_comments = json.loads(comments)
    
    #코멘트 수 
    comment_num = json_comments["result"]["count"]['comment']
    comment_no.append(comment_num)
    
    #코멘트 reply 수
    comment_reply_num = json_comments["result"]["count"]['reply']
    comment_no.append(comment_reply_num)

    return comment_no


#json 파싱
def get_comment(n_url):
    global total_comm
    oid = n_url.split("oid=")[1].split("&")[0]
    aid = n_url.split("aid=")[1]
    header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": n_url,
    }
    
    for i in range(0, len(total_comm)):
        #댓글 jqeury URL
        c_url ='https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery112409540374534082234_1571293027001&lang=ko&country=KR&objectId=news{}%2C{}&categoryId=&pageSize={}&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&userType=&useAltSort=true&replyPageSize={}&moveTo=&sort=favorite&includeAllStatus=true&_=1571293027002'.format(oid, aid, total_comm[0], total_comm[1])
        res = requests.get(c_url, headers=header)
        contents = BeautifulSoup(res.content, "html.parser")
        comments = contents.text.replace('jQuery112409540374534082234_1571293027001(','').replace(');','')

        comment_detail = []

        #json 파일 변환 
        json_comments = json.loads(comments)    
        comments = json_comments["result"]['commentList']
        comment_size = int(json_comments["result"]['count']['comment'])
        print(comment_size)
        
        for j in range(0, comment_size):
            for com in comments:         
                user = com["userName"]
                comment_detail.append(user)

                comment = com["contents"].replace('\n','')
                comment_detail.append(comment)

                sympathy_no = com["sympathyCount"]
                comment_detail.append(sympathy_no)

                antipathy_no = com["antipathyCount"]
                comment_detail.append(antipathy_no)

                com_date = com["regTime"][:10]     
                com_time = com["regTime"][11:19]
                date_time = com_date + ' ' + com_time
                comment_detail.append(date_time)

    return comment_detail
        

while page < 10000:
    print("\n페이지_{}의 크롤링을 시작했습니다".format(page))
    url = "https://search.naver.com/search.naver?where=news&query={}&sort=1&ds={}&de={}&nso=so%3Ar%2Cp%3Afrom{}to{}%2Ca%3A&start={}".format(query, s_date, e_date, s_from, e_to, str(page))
    
    req = requests.get(url, headers=header)
    print("네이버 검색창 페이지_{} URL:".format(page), url)

    #html 가져오기
    html = req.content
    soup = BeautifulSoup(html, 'lxml')

    for urls in soup.select("._sp_each_url"):
        try:
            if urls["href"].startswith("https://news.naver.com"):

                #네이버 뉴스 중 엔터테인먼트 섹션에 게재된 기사 필터팅 (제외)
                if not "&sid1=106" in str(urls["href"]):
                    news_detail = get_news(urls["href"])           #네이버 뉴스 정보 가져오기
                    total_comm = get_comment_num(urls["href"])     #기사의 댓글 건수 가져오기
                    like_statics = get_like_statics(urls["href"])  #기사의 공감 지표(좋아요/슬퍼요 등) 가져오기      
                    if total_comm[0] == 0:                         #댓글 있는 기사에 한하여 댓글 정보 가져오기
                        pass
                    else:
                        naver_comments = get_comment(urls["href"])
                    df_news=df_news.append(Series([query, news_detail[0], news_detail[1], news_detail[2], news_detail[3], news_detail[4], urls["href"], total_comm[0], like_statics[0], like_statics[1],like_statics[2], like_statics[3], like_statics[4]], index=columns_news), ignore_index=True)
                    df_comments=df_comments.append(Series([urls["href"], naver_comments[0], naver_comments[1], naver_comments[2], naver_comments[3], naver_comments[4]], index=columns_comments), ignore_index=True)
            else:
                continue
                
        except Exception as e:
            print(e)
            continue
    #네이버 검색 시, 한 페이지에 기사 10건씩 검색됨
    page += 10

print('='*50)
print("기사 수집을 종료합니다")
print('='*50)


페이지_1의 크롤링을 시작했습니다
네이버 검색창 페이지_1 URL: https://search.naver.com/search.naver?where=news&query=자가격리&sort=1&ds=2020.01.01&de=2020.01.31&nso=so%3Ar%2Cp%3Afrom20200101to20200131%2Ca%3A&start=1
3
3
30
30
28
28
47
47
1
1
17
17

페이지_11의 크롤링을 시작했습니다
네이버 검색창 페이지_11 URL: https://search.naver.com/search.naver?where=news&query=자가격리&sort=1&ds=2020.01.01&de=2020.01.31&nso=so%3Ar%2Cp%3Afrom20200101to20200131%2Ca%3A&start=11
5
5
148
148
154
154
560
560
2
2
115
115
1
1

페이지_21의 크롤링을 시작했습니다
네이버 검색창 페이지_21 URL: https://search.naver.com/search.naver?where=news&query=자가격리&sort=1&ds=2020.01.01&de=2020.01.31&nso=so%3Ar%2Cp%3Afrom20200101to20200131%2Ca%3A&start=21
11
11
507
507
59
59
17
17
13
13
129
129

페이지_31의 크롤링을 시작했습니다
네이버 검색창 페이지_31 URL: https://search.naver.com/search.naver?where=news&query=자가격리&sort=1&ds=2020.01.01&de=2020.01.31&nso=so%3Ar%2Cp%3Afrom20200101to20200131%2Ca%3A&start=31
117
117
1
1
164
164
3
3
154
154
3
3

페이지_41의 크롤링을 시작했습니다
네이버 검색창 페이지_41 URL: https://search.naver.com/search.naver?wher

In [2]:
df_news.head()

,검색어,게재일,게재시간,매체명,헤드라인,본문,URL,댓글 수,좋아요 수,슬퍼요 수,화나요 수,훈훈해요 수,후속기사 희망 수
0,자가격리,2020.01.31,오후 11:58,SBS,'7번 확진자와 접촉' 수원·안양 어린이집 교사들 '음성',신종 코로나바이러스 국내 7번째 확진 환자와 접촉한 것으로 확인된 수원과 안양 어린...,https://news.naver.com/main/read.nhn?mode=LSD&...,3,9,0,0,0,0
1,자가격리,2020.01.31,오후 11:55,연합뉴스,'7번 확진자와 접촉' 수원·안양 어린이집 교사들 '음성',의왕시 거주 20대 자매…만일 상황 대비해 2월 중순까지 자가격리 (수원·안양=연...,https://news.naver.com/main/read.nhn?mode=LSD&...,30,15,0,33,2,1
2,자가격리,2020.01.31,오후 11:30,YTN,국내 첫 3차 감염 발생...확진자 총 11명,"동영상 뉴스 \t \t■ 진행 : 이종구 앵커, 차정윤 앵커■ 출연 : 최재...",https://news.naver.com/main/read.nhn?mode=LSD&...,28,6,0,17,1,0
3,자가격리,2020.01.31,오후 10:56,연합뉴스TV,국내 첫 3차 감염 추정…확진자 11명으로 늘어,동영상 뉴스 \t \t[앵커] 국내에서 처음으로 3차 감염으로 추정되는 신종...,https://news.naver.com/main/read.nhn?mode=LSD&...,47,3,8,44,2,1
4,자가격리,2020.01.31,오후 10:30,뉴스1,김포시 신종 코로나 자가격리 3명→20명 증가(종합),자가·능동감시자 신종 코로나 바이러스 감염증 증상은 없어2 신종 코로나바이러스 감염...,https://news.naver.com/main/read.nhn?mode=LSD&...,1,1,0,5,3,0


In [3]:
df_comments.head()

,URL,댓글작성자,댓글,공감 수,비공감 수,댓글작성시간
0,https://news.naver.com/main/read.nhn?mode=LSD&...,jue7****,다행이네요. 근데 희한하게 하필 많고많은 직업중에 어린이집교사들이 ㅜㅜ,3,1,2020-02-01 01:23:22
1,https://news.naver.com/main/read.nhn?mode=LSD&...,dldl****,7번째확진자도 음성이다해서 격리해제로퇴원시켰지만 그뒤로 안좋아져서다시 확진자로 되었...,102,2,2020-02-01 00:58:57
2,https://news.naver.com/main/read.nhn?mode=LSD&...,dla_****,지금 검색어 조작 아닐까요? 하루종일 확진자 이야기로 가득했는데 지금은 우한 폐렴이...,39,1,2020-02-01 00:07:06
3,https://news.naver.com/main/read.nhn?mode=LSD&...,aimu****,근데 우한폐렴에 걸린 사람들 상태는 어떤지 어느 기사에서도 알수가 없네요 걸렸다는거...,105,1,2020-02-01 00:35:07
4,https://news.naver.com/main/read.nhn?mode=LSD&...,mari****,김포시 어디예요?,6,2,2020-02-01 00:02:46


In [5]:
df_news.to_excel('2020.01.01-2020.01.31_news.xlsx')

In [6]:
df_comments.to_excel('2020.01.01-2020.01.31_comments.xlsx')